In [4]:
!pip install pyESN


In [5]:
!pip install --upgrade --force-reinstall matplotlib


  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/2d/d5/6227732ecab9165586966ccb54301e3164f61b470c954c4cf6940654fbe1/matplotlib-3.8.4-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/d6/4f/76d0dd0bca417691918484c26c74dd9dd44fbf528bbfeb30d754886e2c54/contourpy-1.2.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for cycler>=0.10 from https://files.pythonhosted.org/packages/e7/05/c19819d5e3d95294a6f5947fb9b9629efb316b96de511b418c53d245aae6/cycler-0.12.1-py3-none-any.whl.metadata
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/c5/d4/f426fa1ca42e47bcfff0c878fa9d49d9c03379d00903a7c178f95b97867a/fonttools-4.51.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/162.8 kB ? eta -:--:--
     ----- ---------------------------------- 20.5/162.8 kB ? eta -:-

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
conda-repo-cli 1.0.41 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.41 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.41 requires python-dateutil==2.8.2, but you have python-dateutil 2.9.0.post0 which is incompatible.
conda-repo-cli 1.0.41 requires requests==2.28.1, but you have requests 2.31.0 which is incompatible.
numba 0.57.0 requires numpy<1.25,>=1.21, but you have numpy 1.26.4 which is incompatible.
pycaret 3.2.0 requires matplotlib<=3.6,>=3.3.0, but you have matplotlib 3.8.4 which is incompatible.
pycaret 3.2.0 requires pand

In [6]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pyESN import ESN
from sklearn.metrics import classification_report, confusion_matrix

# Set the path to your dataset
train_data_dir = r"D:\Downloads\Dataset\train"
test_data_dir = r"D:\Downloads\Dataset\test"

# Specify image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Create data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Create data generator for test data
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False)  # Disable shuffling to maintain the order of images

# Reshape the input data to include a time dimension
X_train_reshaped = []
y_train = []

for i in range(len(train_generator)):
    images, labels = train_generator[i]
    for j in range(len(images)):
        X_train_reshaped.append(images[j].reshape(-1))
        y_train.append(labels[j])

X_train_reshaped = np.array(X_train_reshaped)
y_train = np.array(y_train, dtype=np.int32)

# Create and train the ESN
esn = ESN(n_inputs=img_width * img_height * 3, n_outputs=len(np.unique(y_train)), n_reservoir=2000)
esn.fit(X_train_reshaped, y_train)

# Reshape the input data for test data
X_test_reshaped = []
y_test = []

for i in range(len(test_generator)):
    images, labels = test_generator[i]
    for j in range(len(images)):
        X_test_reshaped.append(images[j].reshape(-1))
        y_test.append(labels[j])

X_test_reshaped = np.array(X_test_reshaped)
y_test = np.array(y_test, dtype=np.int32)

# Get the predicted classes for the test data
y_pred = esn.predict(X_test_reshaped)

# Generate the classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Generate the confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


ModuleNotFoundError: No module named 'pyESN'